# Model Controller Tutorial: Roberta model (Regression)

> This notebook contains some example of how to use the Roberta-based models in this NLP library

- skip_showdoc: true
- skip_exec: true

In this series, we walk through some of the capability of this library: single-head classification, multi-head classification, multi-label classification, and regression. If you want a more detailed tutorial, check [this](https://anhquan0412.github.io/that-nlp-library/model_classification_tutorial.html) out

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import os

In [ ]:
#This will specify a (or a list) of GPUs for training
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
from that_nlp_library.text_transformation import *
from that_nlp_library.text_augmentation import *
from that_nlp_library.text_main import *
from that_nlp_library.utils import seed_everything

In [ ]:
from underthesea import text_normalize
from functools import partial
from pathlib import Path
import pandas as pd
import numpy as np
import nlpaug.augmenter.char as nac
from datasets import load_dataset
import random
from transformers import RobertaTokenizer
from datasets import Dataset

# Define the custom augmentation function

In [ ]:
def nlp_aug_stochastic(x,aug=None,p=0.5):
    if not isinstance(x,list): 
        if random.random()<p: return aug.augment(x)[0]
        return x
    news=[]
    originals=[]
    for _x in x:
        if random.random()<p: news.append(_x)
        else: originals.append(_x)
    # only perform augmentation when needed
    if len(news): news = aug.augment(news)
    return news+originals

In [ ]:
aug = nac.KeyboardAug(aug_char_max=3,aug_char_p=0.1,aug_word_p=0.07)
nearby_aug_func = partial(nlp_aug_stochastic,aug=aug,p=0.3)

# Create a TextDataController object

We will reuse the data and the preprocessings in [this tutorial](https://anhquan0412.github.io/that-nlp-library/text_main.html) 

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')


In [ ]:
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Rating',
                         sup_types='regression',
                         filter_dict={'Review Text': lambda x: x is not None},
                         metadatas=['Title','Division Name'],
                         content_transformations=[text_normalize,str.lower],
                         content_augmentations= [nearby_aug_func,str.lower], 
                         # add "str.lower" here because nearby_aug might return uppercase character
                         val_ratio=0.2,
                         batch_size=1000,
                         seed=42,
                         num_proc=20,
                         verbose=False
                        )

Define our tokenizer for Roberta

In [ ]:
_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Process and tokenize our dataset

In [ ]:
tdc.process_and_tokenize(_tokenizer,max_length=100,shuffle_trn=True)

Map (num_proc=20):   0%|          | 0/18112 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/18112 [00:00<?, ? examples/s]

Flattening the indices (num_proc=20):   0%|          | 0/18112 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/18112 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4529 [00:00<?, ? examples/s]

In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Title', 'Review Text', 'Rating', 'Division Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18112
    })
    validation: Dataset({
        features: ['Title', 'Review Text', 'Rating', 'Division Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4529
    })
})

# Model Experiment: Roberta Single-Head Regression

In [ ]:
from that_nlp_library.models.roberta.classifiers import *
from that_nlp_library.model_main import *
from sklearn.metrics import mean_absolute_error,mean_squared_log_error,f1_score, accuracy_score

comet_ml is installed but `COMET_API_KEY` is not set.


## Using HuggingFace model initialization

In [ ]:
from transformers.models.roberta.modeling_roberta import RobertaForSequenceClassification


In [ ]:
num_classes=1

In [ ]:
seed_everything(42)
model = RobertaForSequenceClassification.from_pretrained('roberta-base',num_labels=num_classes)
model = model.to('cuda:0')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric_funcs = [mean_absolute_error,mean_squared_log_error]
controller = ModelController(model,tdc,seed=42)

In [ ]:
lr = 1e-4
bs=32
wd=0.01
epochs= 3

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )


/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mean Absolute Error Rating,Mean Squared Log Error Rating
1,No log,0.396948,0.420621,0.024207
2,1.061100,0.370075,0.394411,0.023088
3,1.061100,0.331583,0.395671,0.020160


## Using Roberta-base model (no concatenation), but with a custom head to limit the output range

In [ ]:
from transformers.models.roberta.modeling_roberta import RobertaModel
import torch

In [ ]:
num_classes=1

In [ ]:
roberta_body = RobertaModel.from_pretrained('roberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class RobertaSigmoidRange(torch.nn.Module):
    def __init__(self,
                 config,
                 high,
                 low,
                 **kwargs
                ):
        super().__init__()
        self.high=high
        self.low=low
        self.score = torch.nn.Linear(config.hidden_size, config.num_labels, bias=False)
    def forward(self, inp, **kwargs):
        logits = self.score(inp)
        return torch.sigmoid(logits)*(self.high-self.low)+self.low

In [ ]:
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': RobertaSigmoidRange,
    # classfication head hyperparams
    'high':5, # the maximum rating
    'low': 1, # the minimum rating
}

model = model_init_classification(model_class = RobertaBaseForSequenceClassification,
                                  cpoint_path = 'roberta-base', 
                                  output_hidden_states=True, # since we are using 'hidden layer contatenation' technique
                                  seed=42,
                                  body_model=roberta_body,
                                  model_kwargs = _model_kwargs)

Loading body weights. This assumes the body is the very first block of your custom architecture


In [ ]:
metric_funcs = [mean_absolute_error,mean_squared_log_error]
controller = ModelController(model,tdc,seed=42)

In [ ]:
seed_everything(42)

lr = 1e-4
bs=32
wd=0.01
epochs= 3

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )


/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mean Absolute Error Rating,Mean Squared Log Error Rating
1,No log,0.389167,0.442911,0.023549
2,0.495000,0.312626,0.392596,0.019879
3,0.495000,0.311144,0.369213,0.019642


## Using the Roberta custom model (concatenation)

In [ ]:
from transformers.models.roberta.modeling_roberta import RobertaModel

In [ ]:
num_classes=1

In [ ]:
roberta_body = RobertaModel.from_pretrained('roberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    # classfication head hyperparams
    'layer2concat':4,
    'classifier_dropout':0.1 
}

In [ ]:
model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                  cpoint_path = 'roberta-base', 
                                  output_hidden_states=True, # since we are using 'hidden layer contatenation' technique
                                  seed=42,
                                  body_model=roberta_body,
                                  model_kwargs = _model_kwargs)

Loading body weights. This assumes the body is the very first block of your custom architecture


In [ ]:
metric_funcs = [mean_absolute_error,mean_squared_log_error]
controller = ModelController(model,tdc,seed=42)

In [ ]:
seed_everything(42)

lr = 1e-4
bs=32
wd=0.01
epochs= 3

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )
# Epoch	Training Loss	Validation Loss	Mean Absolute Error Rating	Mean Squared Log Error Rating
# 1	No log	0.501656	0.478864	0.032053
# 2	0.973400	0.473814	0.484233	0.027219
# 3	0.973400	0.347459	0.405567	0.021133

/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mean Absolute Error Rating,Mean Squared Log Error Rating
1,No log,0.466035,0.510674,0.026711
2,1.170900,0.380608,0.429796,0.022825
3,1.170900,0.331785,0.388653,0.020313


In [ ]:
controller.trainer.model.save_pretrained('./sample_weights/my_model')

## Predict Validation

In [ ]:
_model_kwargs

{'head_class_sizes': 1,
 'head_class': that_nlp_library.models.roberta.classifiers.ConcatHeadSimple,
 'layer2concat': 4,
 'classifier_dropout': 0.1}

In [ ]:
trained_model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                          cpoint_path = Path('./sample_weights/my_model'), 
                                          output_hidden_states=True,
                                          seed=42,
                                          model_kwargs = _model_kwargs)

controller = ModelController(trained_model,tdc,seed=42)

Some weights of the model checkpoint at sample_weights/my_model were not used when initializing RobertaHiddenStateConcatForSequenceClassification: ['body_model.pooler.dense.bias', 'body_model.pooler.dense.weight']
- This IS expected if you are initializing RobertaHiddenStateConcatForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaHiddenStateConcatForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
df_val = controller.predict_ddict(ds_type='validation')

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

In [ ]:
df_val = df_val.to_pandas()
df_val.head()

,Title,Review Text,Rating,Division Name,label,input_ids,attention_mask,pred_Rating
0,,general . . this picture doesn't do the skirt ...,5.0,general,5.0,"[0, 15841, 479, 479, 42, 2170, 630, 75, 109, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5.038281
1,,"general . . easy to wear ! cute , comfy ... wi...",4.0,general,4.0,"[0, 15841, 479, 479, 1365, 7, 3568, 27785, 119...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4.960517
2,,"general . . nice sweater , just did not look g...",3.0,general,3.0,"[0, 15841, 479, 479, 2579, 23204, 2156, 95, 22...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3.172506
3,nice cropped jacket,general . nice cropped jacket . this jacket wa...,5.0,general,5.0,"[0, 15841, 479, 2579, 30197, 8443, 479, 42, 84...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4.224070
4,great dress!,general petite . great dress ! . i wasn't plan...,5.0,general petite,5.0,"[0, 15841, 4716, 1459, 479, 372, 3588, 27785, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5.155542


You can try to get your metric to see if it matches your last traing epoch's above

In [ ]:
mean_absolute_error(df_val['label'],df_val['pred_Rating'])

0.38864555856886224

In [ ]:
mean_squared_log_error(df_val['label'],df_val['pred_Rating'])

0.0203126439014012

## Predict Test set

We will go through details on how to make a prediction on a completely new and raw dataset using our trained model. For now, let's reuse the sample csv and pretend it's our test set

In [ ]:
df_test = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig').sample(frac=0.2,random_state=1)


# save the label, as we will calculate some metrics later
true_labels = df_test[~df_test['Review Text'].isna()].Rating.values 

# drop the label (you don't need to, but this is necessary to simulate an actual test set)
df_test.drop('Rating',axis=1,inplace=True)

In [ ]:
_test_dset = Dataset.from_pandas(df_test)
_test_dset_predicted = controller.predict_raw_dset(_test_dset,
                                                   do_filtering=True, # since we have some text filtering in the processing
                                                  )

Filter (num_proc=20):   0%|          | 0/4697 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4533 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4533 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4533 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4533 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4533 [00:00<?, ? examples/s]

Map:   0%|          | 0/4533 [00:00<?, ? examples/s]

In [ ]:
df_test_predicted = _test_dset_predicted.to_pandas()

In [ ]:
df_test_predicted.head()

,Title,Review Text,Division Name,input_ids,attention_mask,pred_Rating
0,perfect for work and play,general . perfect for work and play . this shi...,general,"[0, 15841, 479, 1969, 13, 173, 8, 310, 479, 42...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5.122227
1,,general petite . . i don't know why i had the ...,general petite,"[0, 15841, 4716, 1459, 479, 479, 939, 218, 75,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4.103627
2,great pants,general petite . great pants . thes e cords ar...,general petite,"[0, 15841, 4716, 1459, 479, 372, 9304, 479, 5,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5.110578
3,surprisingly comfy for a button down,general petite . surprisingly comfy for a butt...,general petite,"[0, 15841, 4716, 1459, 479, 10262, 3137, 24382...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4.751888
4,short and small,general petite . short and small . the shirt i...,general petite,"[0, 15841, 4716, 1459, 479, 765, 8, 650, 479, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2.518289


Let's quickly check the score to make sure everything works correctly

In [ ]:
mean_absolute_error(true_labels,df_test_predicted['pred_Rating'])

0.33805381865814876

In [ ]:
mean_squared_log_error(true_labels,df_test_predicted['pred_Rating'])

0.015574132893257393

In [ ]:
# Since we have some metadatas (Title and Division Name), we need to define a dictionary containing those values
raw_content={'Review Text': 'This shirt is so comfortable I love it!',
             'Title': 'Great shirt',
             'Division Name': 'general'}

In [ ]:
df_result = controller.predict_raw_text(raw_content)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

TypeError: iteration over a 0-d array

In [ ]:
# TODO wtfffff

In [ ]:
df_result

,Review Text,Title,Division Name,input_ids,attention_mask,pred_Department Name,pred_prob_Department Name
0,general . great shirt . this shirt is so comfo...,great shirt,general,"[24622, 764, 1049, 10147, 764, 428, 10147, 318...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[Tops, Jackets, Dresses]","[0.99976474, 8.1968574e-05, 5.9176444e-05]"
